# Functional Connectivity

In this tutorial, we'll cover the basics of functional connectivity analyses. In brief, functional connectivity examines the correlations between timeseries of brain regions. Functional connectivity is often analyzed in the context of resting-state fMRI, which often referred to as 'intrinsic' functional connectivity because these correlations arise in the absence of tasks and behaviour.

We'll analyze a 20 minute resting-state scan of one subject to explore the methodology and types of analyses we can perform. We'll start by getting an _atlas_ that let's us define non-overlapping cortical regions. Then, we'll extract the _mean timeseries_ of these regions using nilearn's `NiftiLabelsMasker`, which is similar to previous masker objects we've used, except that it directly computes the mean timeseries for our regions -- easy! Next, we'll run a functional connectivity analysis by correlating the timeseries between each possible pairs of regions, examine the resulting connectivity matrix, and discuss how we can begin to analyze this matrix.     

**Installing Brain Connectivity Toolbox**

But first we need to install an external dependency! [Brain Connectivity Toolbox](https://sites.google.com/site/bctnet/) (BCT) is a powerful MATLAB toolbox for analyzing functional connectivity. There is a Python version of BCT called `bctpy` and the Github can be found here: https://github.com/aestrivex/bctpy (see the Wiki here: https://github.com/aestrivex/bctpy/wiki). We'll be using functions from `bctpy` to analyze connectivity matrices (actual connectivity matrices are computed using nilearn). 

Unfortunately, this package is fairly niche and is not set up to install using the standard `pip` or `conda` install command. So, you need to download the package from Github directly (`Clone or download` > `Download ZIP`) and **place it in this directory**. Then run the command below:

In [ ]:
pip install -e bctpy-master

Now **restart the notebook kernel** by going to `Kernel` > `Restart Kernel...`. You'll now be able to import `bct`. 
)
In addition to `bct`, we'll also import a few functions/classes from nilearn that will let us first define our regions (`fetch_atlas_schaefer_2018`) and then extract the mean timeseries of these regions (`NiftiLabelsMasker`). We'll also import  `ConnectivityMeasure` from nilearn, which makes it really easy to run a functional connectivity analysis. Finally, we'll import the plotting and image modules entirely rather than their individual functions. 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats, special
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib

from nilearn.datasets import fetch_atlas_schaefer_2018
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn import plotting, image
import bct

%matplotlib inline

## 1. Getting an atlas

Atlases provide a map of brain regions. Atlases can come in a variety of formats, including a labelled 3D NIfTI image (i.e. each voxel labelled according to its region), a 4D NIfTI image in which each volume is a probabilistic map of a region (each voxel is assigned a probability that it belongs to each region), or a list of coordinates that denote the center of each region. 

Nilearn has a [number of functions that let us fetch popular atlases](https://nilearn.github.io/modules/reference.html#module-nilearn.datasets). We'll use one of these functions to fetch the Schaefer altas ([link to the paper](https://academic.oup.com/cercor/article/28/9/3095/3978804), which is a recent atlas that determines region boundaries based on sudden changes in voxelwise functional connectivity (called 'gradients') and clustering approaches. 

In [ ]:
atlas = fetch_atlas_schaefer_2018(n_rois=100, resolution_mm=2)

# we can get the image out of the dictionary
atlas_img = nib.load(atlas['maps'])

We can plot the atlas overlaid on top of a MNI template. The coordinates I've selected highlight some nice features of the atlas:

In [ ]:
plotting.plot_roi(atlas_img, cmap='jet', cut_coords=[6, 20, 50])

Alternatively, you can explore the atlas interactively:

In [ ]:
plotting.view_img(atlas_img, cmap='jet', symmetric_cmap=False)

We can also get the region labels, in order of numerical label:

In [ ]:
# decode just converts it from byte-formate to string
labels = [x.decode() for x in atlas['labels']]
labels

## 2. Extracting regions timeseries from resting state

Now that we have an atlas, we want to extract the mean timeseries from each region. We also want to apply some post-processing to these timeseries so that we can remove low-frequency trends in the data (temporal filtering) and regress out sources of noise (i.e. confound regression). 

Although this may seem daunting (we have 200 regions), it is actually really easy to do this all in one step using nilearn

First, we'll load in our confounds that we wish to regress out. Remember that confounds are found in `regressor_confounds.tsv` files produced by fmriprep. Instead of loading in the whole file, we'll only load in the confounds that we want. You can see that I have specified more confounds than we are used to seeing (only the 6 motion parameters). We are going to get the 6 motion parameters, their derivatives, their square, and the derivatives of the squares (24 motion parameters total, commonly called Friston24). We'll also include the framewise displacement (a composite measure of motion), and the mean signals of white matter and CSF. The two latter confounds give us signal fluctuations that we presume to be physiological noise that we want to remove from our data.

Try also adding 'global_signal' by uncommenting the final line in the cell below:

In [ ]:
confound_names = ['trans_x', 'trans_y', 'trans_z',
                  'trans_x_derivative1', 'trans_y_derivative1', 'trans_z_derivative1',
                  'trans_x_power2', 'trans_y_power2', 'trans_z_power2',
                  'trans_x_derivative1_power2', 'trans_y_derivative1_power2', 'trans_z_derivative1_power2',
                  'rot_x', 'rot_y', 'rot_z', 
                  'rot_x_derivative1', 'rot_y_derivative1', 'rot_z_derivative1',
                  'rot_x_power2', 'rot_y_power2', 'rot_z_power2',
                  'rot_x_derivative1_power2', 'rot_y_derivative1_power2', 'rot_z_derivative1_power2', 
                  'framewise_displacement', 'csf', 'white_matter']

# confound_names.append('global_signal')

In [ ]:
confounds = pd.read_table('rs-data/sub-01_task-rest_run-03_desc-confounds_regressors.tsv', usecols=confound_names)
confounds.head()

Note that the first row has not-a-number (`NaN`) for the derivatives; this is unavoidable because derivatives are the rate of change between data points, so it requires two datapoints by definition. Nilearn unfortunately, but understandably, does not accept `NaN`s so we need to drop that row. This row corresponds to the first functional volume, which now needs to be dropped as well.

Now let's load in our functional data, and then drop the first volume:

In [ ]:
func_img = nib.load('rs-data/sub-01_task-rest_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
func_img.shape

In [ ]:
# drop first volume from image
func_img = image.index_img(func_img, slice(1, None))

func_img.shape

And let's not forget to drop the first row of our confounds:

In [ ]:
# convert confounds to numpy array (from DataFrame), and drop first row
confounds = confounds.values[1:, :]

Now we are ready to extract data. We will use `NiftiLabelsMasker` post-process, apply confound regression, and extract the mean timeseries of each region. This function is very similar to what we have seen before. The result will be a time by region array:

In [ ]:
# set up our parameters
masker = NiftiLabelsMasker(labels_img=atlas_img, high_pass=.01, detrend=True, standardize=True, 
                           t_r=2)
# extract
data = masker.fit_transform(func_img, confounds=confounds)

Now let's see the time by region array:

In [ ]:
data.shape

In [ ]:
plt.figure(figsize=(5, 12))
plt.imshow(data, aspect='auto')

Above, each row is a timepoint (or volume), and each column is a region. You can see fluctuations of activity at certain timepoints. Some of these fluctuations seem to be shared by only some regions. 

## 3. Computing functional connectivity

### 3.1 Example correlation

Functional connectivity is essentially computing correlations between each column in the matrix. For example:

In [ ]:
# get two regions and compute correlation
region1 = data[:, 0]
region2 = data[:, 1]
r, p = stats.pearsonr(region1, region2)

# multipanel figure (axes is now an array)
fig, axes = plt.subplots(ncols=2, figsize=(12, 3))
# unpack axes
ax1, ax2 = axes
# timeseries plot
ax1.plot(range(data.shape[0]), region1) # blue
ax1.plot(range(data.shape[0]), region2) # orange
ax1.set(xlabel='Volume', ylabel='Signal (z)', title='Timeseries')
# correlation plot
ax2.scatter(region1, region2, alpha=.5, c='C7')
ax2.set(xlabel='Region 1 (z)', ylabel='Region 2 (z)', title=f'r = {r:.3f}');

On the left, we see the timeseries for both regions. The right shows the scatter plot of both region, where each datapoint is a volume/timepoint. 

### 3.2 Correlation or connectivity Matrix

We can easily expand this to all possible combinations of correlations in our data by using nilearn's `ConnectivityMeasure` ([see documentation](https://nilearn.github.io/modules/generated/nilearn.connectome.ConnectivityMeasure.html#nilearn.connectome.ConnectivityMeasure)). Calling the `fit_transform` method, as we've seen before, returns a _connectivity matrix_ showing each pairwise correlation. 

In [ ]:
connect = ConnectivityMeasure(kind='correlation')

# analyze our matrix as a list because ConnectivityMeasure expects multi-subject data
cmat = connect.fit_transform([data])
# get the connectivity matrix out of the list
cmat = cmat[0]

# show the shape
cmat.shape

Now we can plot this matrix:

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(cmat, labels=labels, figure=fig, vmin=-1, vmax=1)

We can see that this is a 200x200 connectivity matrix, and the overwhelming majority of correlations are positive. Let's take a look at a submatrix to break this large matrix down and understand how to interpret it. We'll take the first 9 regions, which correspond to the left visual network:

In [ ]:
left_vis = cmat[:9, :9]

fig, ax = plt.subplots(figsize=(4, 4))
plotting.plot_matrix(left_vis, labels=labels[:9], figure=fig, vmin=-1, vmax=1)

In [ ]:
vis_img = image.math_img("np.where(np.isin(img, np.arange(1, 10)), img, 0)", img=atlas_img)
plotting.plot_roi(vis_img, vmax=10, vmin=1, cmap='tab10', colorbar=True,
                  cut_coords=[0, -10, -20, -30, -40, -50], display_mode='x')

It now makes total sense as to why `17Networks_LH_Vis_1` is so different from the rest; it is really part of the parahippocampus rather than visual cortex. 

### 3.3 Sorting a connectivity matrix

Returning to the full connectivity matrix, we can also see some networks emerge, as well as strong interhemispheric connectivity (shown by large 'squares' in the off-diagonal). Like we did with RSA representational dissimilarity matrices, we can also show a connectivity map that has been clustered based on the correlations. This essentially sorts our connectivity matrix into networks:

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(cmat, labels=labels, figure=fig, vmin=-1, vmax=1, reorder=True)

## 4. Thresholding

Often we want retain correlations that fall above a certain threshold. We can threshold matrices in a variety of different ways. Of course, you may choose to convert your _r_ values to _p_ values, and then apply statistical thresholding (e.g., _p_ < .05 or _p_ < .01) and then do some sort of multiple comparisons correction (e.g., false-discovery rate correction).

### 4.1 Absolute thresholding

A more conventional and simpler approach is to threshold at _r_ = .3. I'm not totally sure _why_ this is the cutoff, but it is sensible because _r_ values above .3 are generally considered to be moderate effect size. In our case here, _r_ > .3 corresponds to $p < 10^{-14}$ (where _n_ = 611, the number of volumes/timepoints). To do this, we can use `threshold_absolute` from `bctpy`:  

In [ ]:
threshold = .3
thresh_mat = bct.threshold_absolute(cmat, threshold)

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(thresh_mat, labels=atlas['labels'], figure=fig, vmin=-1, vmax=1)

### 4.2 Proportional thresholding

We can also threshold based on the top proportion of correlations. For instance, we can select the correlations that fall within the 80th percentile (i.e. top 20%). We can use the `threshold_proportional` function from `bctpy`:

In [ ]:
threshold = .2
thresh_mat = bct.threshold_proportional(cmat, threshold)

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(thresh_mat, labels=atlas['labels'], figure=fig, vmin=-1, vmax=1)

This is much sparser, which may be useful for certain applications.

### 4.3 Binarizing a thresholded matrix

We can also binarize our connectivity matrix (convert every nonzero value to 1). Binarization is necessary for some graph theoretical methods that we'll explore in the next tutorial. We can call the `binarize` function on one of our thresholded matrices.

In [ ]:
binary_mat = bct.binarize(thresh_mat)

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(binary_mat, labels=atlas['labels'], figure=fig, cmap='binary', colorbar=False)